<a href="https://colab.research.google.com/github/sakethgoud321/Forecasting/blob/main/Books_Recommendation_System_Model_Building__(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Age of Recommender Systems

The rapid growth of data collection has led to a new era of information. Data is being used to create more efficient systems and this is where Recommendation Systems come into play. Recommendation Systems are a type of information filtering systems as they improve the quality of search results and provides items that are more relevant to the search item or are realted to the search history of the user.

They are used to predict the rating or preference that a user would give to an item. Almost every major tech company has applied them in some form or the other: Amazon uses it to suggest products to customers, YouTube uses it to decide which video to play next on autoplay, and Facebook uses it to recommend pages to like and people to follow. Moreover, companies like Amazon, Netflix and Spotify depend highly on the effectiveness of their recommendation engines for their business and sucees.

![image.png](attachment:image.png)

### There are basically three types of recommender systems:-

> *  **Demographic Filtering**- They offer generalized recommendations to every user, based on movie popularity and/or genre. The System recommends the same movies to users with similar demographic features. Since each user is different , this approach is considered to be too simple. The basic idea behind this system is that movies that are more popular and critically acclaimed will have a higher probability of being liked by the average audience.

> *  **Content Based Filtering**- They suggest similar items based on a particular item. This system uses item metadata, such as genre, director, description, actors, etc. for movies, to make these recommendations. The general idea behind these recommender systems is that if a person liked a particular item, he or she will also like an item that is similar to it.

> *  **Collaborative Filtering**- This system matches persons with similar interests and provides recommendations based on this matching. Collaborative filters do not require item metadata like its content-based counterparts.


In [ ]:
!pip install nltk
!pip install dataprep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 56.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 676.8 kB/s eta 0:00:00
   

In [ ]:
#importing necessary libraries
import os
import re
import nltk
import requests
import warnings
import pandas as pd
import numpy as np
from math import ceil
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

from PIL import Image
warnings.filterwarnings('ignore')
from dataprep.eda import plot, plot_correlation, plot_missing, create_report

## Importing Dataset

In [ ]:
books_df = pd.read_csv("final_data.csv")

In [ ]:
books_df

,userid,location,age,country,isbn,rating,title,author,year,publisher,image_url_m
0,2,"stockton, california, usa",18.0,usa,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,8,"timmins, ontario, canada",32.0,canada,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,8,"timmins, ontario, canada",32.0,canada,0060973129,0,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,8,"timmins, ontario, canada",32.0,canada,0374157065,0,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,8,"timmins, ontario, canada",32.0,canada,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...,...,...,...
39879,11676,"n/a, n/a, n/a",32.0,None,0760754306,0,Come on Seabiscuit!,Ralph Moody,2004,Barnes &amp; Noble,http://images.amazon.com/images/P/0760754306.0...
39880,11676,"n/a, n/a, n/a",32.0,None,0761105417,8,The Good Luck Book,Stefan Bechtel,1997,Workman Publishing,http://images.amazon.com/images/P/0761105417.0...
39881,11676,"n/a, n/a, n/a",32.0,None,076111940X,8,How to Make People Like You in 90 Seconds or Less,Nicholas Boothman,2000,Workman Publishing,http://images.amazon.com/images/P/076111940X.0...
39882,11676,"n/a, n/a, n/a",32.0,None,0761119779,7,The Pinky Ball Book &amp; the Pinky Ball (Ages...,Dina Anastasio,2000,Workman Publishing,http://images.amazon.com/images/P/0761119779.0...


In [ ]:
import re

books_df['title'] = books_df['title'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

In [ ]:
# explicit data
explicit=books_df[books_df['rating']!=0]
explicit=explicit.reset_index(drop=True)
print(explicit.shape)
explicit.head()

(18602, 11)


,userid,location,age,country,isbn,rating,title,author,year,publisher,image_url_m
0,8,"timmins, ontario, canada",32.0,canada,0002005018,5,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
1,8,"timmins, ontario, canada",32.0,canada,074322678X,5,Where Youll Find Me And Other Stories,Ann Beattie,2002,Scribner,http://images.amazon.com/images/P/074322678X.0...
2,8,"timmins, ontario, canada",32.0,canada,0887841740,5,The Middle Stories,Sheila Heti,2004,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...
3,8,"timmins, ontario, canada",32.0,canada,1552041778,5,Jane Doe,R. J. Kaiser,1999,Mira Books,http://images.amazon.com/images/P/1552041778.0...
4,8,"timmins, ontario, canada",32.0,canada,1567407781,6,The Witchfinder Amos Walker Mystery Series,Loren D. Estleman,1998,Brilliance Audio - Trade,http://images.amazon.com/images/P/1567407781.0...


In [ ]:
# Implicit data
implicit=books_df[books_df['rating']==0]
implicit=implicit.reset_index(drop=True)
print(implicit.shape)
implicit.head()

(21282, 11)


,userid,location,age,country,isbn,rating,title,author,year,publisher,image_url_m
0,2,"stockton, california, usa",18.0,usa,0195153448,0,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,8,"timmins, ontario, canada",32.0,canada,0060973129,0,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
2,8,"timmins, ontario, canada",32.0,canada,0374157065,0,Flu The Story of the Great Influenza Pandemic ...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
3,8,"timmins, ontario, canada",32.0,canada,0393045218,0,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
4,8,"timmins, ontario, canada",32.0,canada,0399135782,0,The Kitchen Gods Wife,Amy Tan,1991,Putnam Pub Group,http://images.amazon.com/images/P/0399135782.0...


## Recommendation System - Part I

### Product popularity based recommendation system targeted at new customers
Popularity based are a great strategy to target the new customers with the most popular products sold on a business's website and is very useful to cold start a recommendation engine.

### Popularity Based Recommender System

In [ ]:
explicit.title.value_counts().reset_index()

,index,title
0,The Da Vinci Code,27
1,The Lovely Bones A Novel,25
2,The Secret Life of Bees,25
3,Wild Animus,22
4,Life of Pi,20
...,...,...
13620,For Whom the Bell Tolls,1
13621,The LAST PICTURE SHOW A Novel,1
13622,True At First Light A Fictional Memoir,1
13623,Give Me My Fathers Body The Life of Minik the...,1


In [ ]:
ratings = pd.DataFrame(explicit.groupby('title')['rating'].mean())
ratings.rename({'rating':'avg_ratings'}, axis=1 , inplace =True)
ratings['num_ratings'] = pd.DataFrame(explicit.groupby('title')['rating'].count())
ratings.reset_index(inplace=True)
ratings

,title,avg_ratings,num_ratings
0,Sie belieben wohl zu scherzen Mr Feynman Aben...,9.0,1
1,010100 A Novel of the Millennium,8.0,1
2,010100 The Novel of the Millennium,6.0,1
3,1 2 3 Music,8.0,1
4,10 Minute Guide to HTML 32,7.0,1
...,...,...,...
13620,Zwlf Geschichten aus der Fremde,7.0,1
13621,de Parte de La Princesa Muerta,7.0,1
13622,metamorfosis La Clsicos seleccin series,8.0,1
13623,stardust,8.0,2


In [ ]:
popular_books = ratings[ratings['num_ratings']>=100].sort_values('avg_ratings', ascending=False).head(10)
popular_books.reset_index(inplace=True, drop=True)
popular_books

,title,avg_ratings,num_ratings


In [ ]:
popular_books.sort_values('num_ratings', ascending=False)

,title,avg_ratings,num_ratings


In [ ]:
twentyfive_popular_books = popular_books.merge(explicit, on='title').drop_duplicates('title').reset_index(drop=True)
twentyfive_popular_books = twentyfive_popular_books[['title','avg_ratings','num_ratings','author','publisher','image_url_m']]
twentyfive_popular_books

,title,avg_ratings,num_ratings,author,publisher,image_url_m


In [ ]:
twentyfive_popular_books

,title,avg_ratings,num_ratings,author,publisher,image_url_m


In [ ]:
def show_image(val):
    return '<img src="{}" width=50></img'.format(val)

In [ ]:
popular_books_t_a_w = twentyfive_popular_books[['title','author','image_url_m']]
popular_books_t_a_w = popular_books_t_a_w.style.format({'image_url_m':show_image})

In [ ]:
popular_books_t_a_w

,title,author,image_url_m


In [ ]:
from PIL import Image
import requests

In [ ]:
filteredImages = twentyfive_popular_books['image_url_m'].tolist()
caption = twentyfive_popular_books['title'].tolist()

## Pre Proessing

### Data Cleaning

In [ ]:
# Removing Users who have rated less than 50 Books
user_counts = pd.DataFrame(explicit.groupby('userid').count()['rating']).rename({'rating':'no_times_rated'}, axis=1).reset_index()
user_counts

,userid,no_times_rated
0,8,7
1,9,1
2,12,1
3,14,3
4,16,1
...,...,...
2715,11662,1
2716,11667,1
2717,11668,1
2718,11675,1


In [ ]:
final_books = pd.merge(explicit, user_counts, on='userid')
final_books.drop(columns = ['location','isbn',
                   'country','age',
                   'year'],axis=1,inplace = True) #remove useless cols
final_books.head()

,userid,rating,title,author,publisher,image_url_m,no_times_rated
0,8,5,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,7
1,8,5,Where Youll Find Me And Other Stories,Ann Beattie,Scribner,http://images.amazon.com/images/P/074322678X.0...,7
2,8,5,The Middle Stories,Sheila Heti,House of Anansi Press,http://images.amazon.com/images/P/0887841740.0...,7
3,8,5,Jane Doe,R. J. Kaiser,Mira Books,http://images.amazon.com/images/P/1552041778.0...,7
4,8,6,The Witchfinder Amos Walker Mystery Series,Loren D. Estleman,Brilliance Audio - Trade,http://images.amazon.com/images/P/1567407781.0...,7


In [ ]:
final_books.userid.nunique()

2720

In [ ]:
final_books = final_books[final_books['no_times_rated']>=10].reset_index(drop=True)
final_books

,userid,rating,title,author,publisher,image_url_m,no_times_rated
0,242,10,The Martian Chronicles,RAY BRADBURY,Spectra,http://images.amazon.com/images/P/0553278223.0...,22
1,242,10,Kabale Und Liebe,Schiller,"Philipp Reclam, Jun Verlag GmbH",http://images.amazon.com/images/P/3150000335.0...,22
2,242,9,Der illustrierte Mann Erzhlungen,Ray Bradbury,Diogenes,http://images.amazon.com/images/P/3257203659.0...,22
3,242,10,Der Knig in Gelb,Raymond Chandler,Diogenes Verlag,http://images.amazon.com/images/P/3257207522.0...,22
4,242,8,Fahrenheit 451,Ray Bradbury,Distribooks Inc,http://images.amazon.com/images/P/3257208626.0...,22
...,...,...,...,...,...,...,...
13751,11676,10,The Call of the Wild Barnes amp Noble Classics,Jack London,Barnes &amp; Noble Books,http://images.amazon.com/images/P/0760727937.0...,5307
13752,11676,8,The Good Luck Book,Stefan Bechtel,Workman Publishing,http://images.amazon.com/images/P/0761105417.0...,5307
13753,11676,8,How to Make People Like You in 90 Seconds or Less,Nicholas Boothman,Workman Publishing,http://images.amazon.com/images/P/076111940X.0...,5307
13754,11676,7,The Pinky Ball Book amp the Pinky Ball Ages 5Up,Dina Anastasio,Workman Publishing,http://images.amazon.com/images/P/0761119779.0...,5307


In [ ]:
final_books.userid.nunique()

236

In [ ]:
final_books.sort_values('no_times_rated', ascending=False)

,userid,rating,title,author,publisher,image_url_m,no_times_rated
13755,11676,10,The Breast Book An Intimate and Curious History,Maura Spiegel,NaN,NaN,5307
10393,11676,5,The Far Side Observer,Gary Larson,Andrews McMeel Publishing,http://images.amazon.com/images/P/0836220986.0...,5307
10329,11676,9,Three Lives Stories of the Good Anna Melanctha...,Gertrude Stein,Dover Publications,http://images.amazon.com/images/P/0486280594.0...,5307
10330,11676,7,The Fourth Hand,John Irving,Ballantine Books,http://images.amazon.com/images/P/0345449347.0...,5307
10331,11676,9,Straight Man A Novel Vintage Contemporaries,RICHARD RUSSO,Vintage,http://images.amazon.com/images/P/0375701907.0...,5307
...,...,...,...,...,...,...,...
8099,10946,5,Time and Chance,Alan Brennert,Tor Books,http://images.amazon.com/images/P/0812531884.0...,10
8098,10946,5,Remember When,Judith McNaught,Pocket,http://images.amazon.com/images/P/0671795554.0...,10
8097,10946,5,Secret Honor Honor Bound Paperback,W. E. B. Griffin,Jove Books,http://images.amazon.com/images/P/0515130095.0...,10
8096,10946,5,The Last Sanctuary,Craig Holden,Dell Publishing Company,http://images.amazon.com/images/P/0440217334.0...,10


In [ ]:
final_books.duplicated().sum()

0

In [ ]:
final_books.isnull().sum()

userid            0
rating            0
title             0
author            0
publisher         1
image_url_m       1
no_times_rated    0
dtype: int64

### Removing Books with less than 50 number of User Ratings

In [ ]:
final_book = final_books[final_books['no_times_rated'] >= 50]
final_book

,userid,rating,title,author,publisher,image_url_m,no_times_rated
40,254,8,The Dark Half,Stephen King,Signet Book,http://images.amazon.com/images/P/0451167317.0...,57
41,254,9,Harry Potter and the Chamber of Secrets Book 2,J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439064864.0...,57
42,254,7,Shes Come Undone Oprahs Book Club,Wally Lamb,Pocket,http://images.amazon.com/images/P/0671021001.0...,57
43,254,9,The Secret Life of Bees,Sue Monk Kidd,Penguin Books,http://images.amazon.com/images/P/0142001740.0...,57
44,254,9,Harry Potter and the Chamber of Secrets Book 2,J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439064872.0...,57
...,...,...,...,...,...,...,...
13751,11676,10,The Call of the Wild Barnes amp Noble Classics,Jack London,Barnes &amp; Noble Books,http://images.amazon.com/images/P/0760727937.0...,5307
13752,11676,8,The Good Luck Book,Stefan Bechtel,Workman Publishing,http://images.amazon.com/images/P/0761105417.0...,5307
13753,11676,8,How to Make People Like You in 90 Seconds or Less,Nicholas Boothman,Workman Publishing,http://images.amazon.com/images/P/076111940X.0...,5307
13754,11676,7,The Pinky Ball Book amp the Pinky Ball Ages 5Up,Dina Anastasio,Workman Publishing,http://images.amazon.com/images/P/0761119779.0...,5307


In [ ]:
final_book.duplicated().sum()

0

In [ ]:
df = final_book.copy()
df

,userid,rating,title,author,publisher,image_url_m,no_times_rated
40,254,8,The Dark Half,Stephen King,Signet Book,http://images.amazon.com/images/P/0451167317.0...,57
41,254,9,Harry Potter and the Chamber of Secrets Book 2,J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439064864.0...,57
42,254,7,Shes Come Undone Oprahs Book Club,Wally Lamb,Pocket,http://images.amazon.com/images/P/0671021001.0...,57
43,254,9,The Secret Life of Bees,Sue Monk Kidd,Penguin Books,http://images.amazon.com/images/P/0142001740.0...,57
44,254,9,Harry Potter and the Chamber of Secrets Book 2,J. K. Rowling,Scholastic,http://images.amazon.com/images/P/0439064872.0...,57
...,...,...,...,...,...,...,...
13751,11676,10,The Call of the Wild Barnes amp Noble Classics,Jack London,Barnes &amp; Noble Books,http://images.amazon.com/images/P/0760727937.0...,5307
13752,11676,8,The Good Luck Book,Stefan Bechtel,Workman Publishing,http://images.amazon.com/images/P/0761105417.0...,5307
13753,11676,8,How to Make People Like You in 90 Seconds or Less,Nicholas Boothman,Workman Publishing,http://images.amazon.com/images/P/076111940X.0...,5307
13754,11676,7,The Pinky Ball Book amp the Pinky Ball Ages 5Up,Dina Anastasio,Workman Publishing,http://images.amazon.com/images/P/0761119779.0...,5307


### Recommendation System - Part II
#### Model-based collaborative filtering system
Recommend items to users based on purchase history and similarity of ratings provided by other users who bought items to that of a particular customer.
A model based collaborative filtering technique is chosen here as it helps in recommending products for a particular user by identifying patterns based on preferences from multiple user’s data.

#### Utility Matrix based on products sold and user reviews
**Utility Matrix :** 

An utility matrix is consists of all possible user-item preferences (ratings) details represented as a matrix. The utility matrix is sparce as none of the users would buy all the items in the list, hence, most of the values are unknown.

In [ ]:
pivot_tables = df.pivot_table(index='title', columns='userid',values='rating').fillna(0)
pivot_tables

userid,254,638,1424,1733,1903,2033,2110,2276,2766,2891,...,9417,9613,9856,10314,10354,10447,10560,10819,11601,11676
title,,,,,,,,,,,,,,,,,,,,,
Sie belieben wohl zu scherzen Mr Feynman Abenteuer eines neugierigen Physikers,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010100 A Novel of the Millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
010100 The Novel of the Millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1 2 3 Music,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
10 Minute Guide to HTML 32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombification Stories from National Public Radio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0
Zoya,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
Zoyas Story An Afghan Womans Struggle for Freedom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Compute cosine similarity matrix
cos_sim_matrix = cosine_similarity(pivot_tables.values)
cos_sim_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 1., 1.],
       [0., 1., 1., ..., 0., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 1., 1.],
       [0., 1., 1., ..., 0., 1., 1.]])

In [ ]:
# Create a DataFrame from the similarity matrix
cos_sim_df = pd.DataFrame(cos_sim_matrix, index=pivot_tables.index, columns=pivot_tables.index)
cos_sim_df

title,Sie belieben wohl zu scherzen Mr Feynman Abenteuer eines neugierigen Physikers,010100 A Novel of the Millennium,010100 The Novel of the Millennium,1 2 3 Music,10 Minute Guide to HTML 32,100 Greatest Women,10000 Things to Praise God for,1001 Brilliant Ways to Checkmate,1001 Things Everyone Should Know About Science,1003 Great Things About Kids,...,Zia,Ziggy Faces Life Again,Zodiac The EcoThriller,Zombie,Zombies of the Gene Pool,Zombification Stories from National Public Radio,Zoya,Zoyas Story An Afghan Womans Struggle for Freedom,stardust,street bible the
title,,,,,,,,,,,,,,,,,,,,,
Sie belieben wohl zu scherzen Mr Feynman Abenteuer eines neugierigen Physikers,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010100 A Novel of the Millennium,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.668965,0.0,0.0,0.0,1.0,0.0,1.0,1.0
010100 The Novel of the Millennium,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.668965,0.0,0.0,0.0,1.0,0.0,1.0,1.0
1 2 3 Music,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.668965,0.0,0.0,0.0,1.0,0.0,1.0,1.0
10 Minute Guide to HTML 32,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zombification Stories from National Public Radio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
Zoya,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.668965,0.0,0.0,0.0,1.0,0.0,1.0,1.0
Zoyas Story An Afghan Womans Struggle for Freedom,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
print(pivot_tables.index)

Index([' Sie belieben wohl zu scherzen Mr Feynman Abenteuer eines neugierigen Physikers',
       '010100 A Novel of the Millennium',
       '010100 The Novel of the Millennium', '1 2 3 Music',
       '10 Minute Guide to HTML 32', '100 Greatest Women',
       '10000 Things to Praise God for', '1001 Brilliant Ways to Checkmate',
       '1001 Things Everyone Should Know About Science',
       '1003 Great Things About Kids',
       ...
       'Zia', 'Ziggy Faces Life Again', 'Zodiac The EcoThriller', 'Zombie',
       'Zombies of the Gene Pool',
       'Zombification Stories from National Public Radio', 'Zoya',
       'Zoyas Story An Afghan Womans Struggle for Freedom', 'stardust',
       'street bible the'],
      dtype='object', name='title', length=8520)


In [ ]:
# remove leading and trailing whitespaces from the titles
pivot_tables.index = pivot_tables.index.str.strip()

# check if the title is present in the index
np.where(pivot_tables.index == 'A Nutshell handbook')


(array([], dtype=int64),)

In [ ]:
print(pivot_tables.index)

Index(['Sie belieben wohl zu scherzen Mr Feynman Abenteuer eines neugierigen Physikers',
       '010100 A Novel of the Millennium',
       '010100 The Novel of the Millennium', '1 2 3 Music',
       '10 Minute Guide to HTML 32', '100 Greatest Women',
       '10000 Things to Praise God for', '1001 Brilliant Ways to Checkmate',
       '1001 Things Everyone Should Know About Science',
       '1003 Great Things About Kids',
       ...
       'Zia', 'Ziggy Faces Life Again', 'Zodiac The EcoThriller', 'Zombie',
       'Zombies of the Gene Pool',
       'Zombification Stories from National Public Radio', 'Zoya',
       'Zoyas Story An Afghan Womans Struggle for Freedom', 'stardust',
       'street bible the'],
      dtype='object', name='title', length=8520)


In [ ]:
similarity = cosine_similarity(pivot_tables)
similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 1., 1.],
       [0., 1., 1., ..., 0., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 1., 1., ..., 0., 1., 1.],
       [0., 1., 1., ..., 0., 1., 1.]])

In [ ]:
sorted(list(enumerate(similarity[0])),key = lambda x:x[1], reverse=True)[1:21]

[(880, 1.0),
 (1097, 1.0),
 (1262, 1.0),
 (1585, 1.0),
 (1721, 1.0),
 (1722, 1.0),
 (1723, 1.0),
 (1729, 1.0),
 (1765, 1.0),
 (1766, 1.0),
 (1769, 1.0),
 (1804, 1.0),
 (1830, 1.0),
 (1998, 1.0),
 (1999, 1.0),
 (2307, 1.0),
 (2337, 1.0),
 (2867, 1.0),
 (2923, 1.0),
 (3429, 1.0)]

In [ ]:
list(enumerate(similarity[0]))[1:16]

[(1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0)]

In [ ]:
# Defining a Function
def recommend(book_name):
    index = np.where(pivot_tables.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity[index])),key = lambda x:x[1], reverse=True)[1:16]
    
    for i in similar_items:
        print(pivot_tables.index[i[0]])

In [ ]:
recommend('Forgiving')

010100 The Novel of the Millennium
1 2 3 Music
100 Greatest Women
1003 Great Things About Kids
101 Bug Jokes
101 Dalmatians Proud to Be a Pup
101 Reykjavik
13 99 Euros
13 Ghosts Strange But True Ghost Stories
14000 Things to Be Happy About
1401 More Things That PSs Me Off
19 Purchase St
2001 a Space Odyssey
253
36 Hours Christmas Silhouette Promo


In [ ]:
recommend('Catch Kitten Kitten Board Books')

010100 The Novel of the Millennium
1 2 3 Music
100 Greatest Women
1003 Great Things About Kids
101 Bug Jokes
101 Dalmatians Proud to Be a Pup
101 Reykjavik
13 99 Euros
13 Ghosts Strange But True Ghost Stories
14000 Things to Be Happy About
1401 More Things That PSs Me Off
19 Purchase St
2001 a Space Odyssey
253
36 Hours Christmas Silhouette Promo


In [ ]:
recommend('1984')

Animal Farm
The Fellowship of the Ring The Lord of the Rings Part 1
Stardust
Martian Chronicles
Neverwhere
Stranger In the Mirror
Disclosure
Scandal in Fair Haven
Silver Scream  A BedandBreakfast Mystery BedAndBreakfast Mysteries Paperback
Rebecca
The Indian in the Cupboard rack Indian in the Cupboard
The Charm School
A Murderous Yarn Needlecraft Mysteries
Hello Darling Are You Working
Little Drummer Girl


In [ ]:
def recommend(book_name):
    pivot_tables = df.pivot_table(index='title', columns='userid',values='rating').fillna(0)
    index = np.where(pivot_tables.index==book_name)[0][0]
    similarity = cosine_similarity(pivot_tables)
    similar_items = sorted(list(enumerate(similarity[index])),key = lambda x:x[1], reverse=True)[1:20]
    
    for i in similar_items:
        print(pivot_tables.index[i[0]])

In [ ]:
recommend('1984')

Animal Farm
The Fellowship of the Ring The Lord of the Rings Part 1
Stardust
Martian Chronicles
Neverwhere
Stranger In the Mirror
Disclosure
Scandal in Fair Haven
Silver Scream  A BedandBreakfast Mystery BedAndBreakfast Mysteries Paperback
Rebecca
The Indian in the Cupboard rack Indian in the Cupboard
The Charm School
A Murderous Yarn Needlecraft Mysteries
Hello Darling Are You Working
Little Drummer Girl
Angelas Ashes MMP  A Memoir
All Creatures Great and Small
Growing Pains of Adrian Mole
From Potters Field


In [ ]:
def recommend(book_name):
    pivot_tables = df.pivot_table(index='title', columns='userid',values='rating').fillna(0)
    index = np.where(pivot_tables.index==book_name)[0][0]
    similarity = cosine_similarity(pivot_tables)
    similar_items = sorted(list(enumerate(similarity[index])),key = lambda x:x[1], reverse=True)[1:20]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = df[df['title'] == pivot_tables.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('title')['title'].values))
        item.extend(list(temp_df.drop_duplicates('title')['author'].values))
        item.extend(list(temp_df.drop_duplicates('title')['image_url_m'].values))
        
        data.append(item)
    return data

In [ ]:
book_name = input("Enter book_name:")
if df['title'].str.contains(book_name).any():
    try:
        recommend_books_t_a_w = pd.DataFrame(recommend(book_name),columns=['title', 'author','image'])
        recommend_books_t_a_w = recommend_books_t_a_w.style.format({'image':show_image})
    except IndexError:
        print("Enter the Title which is available")

Enter book_name:The Dark Half


In [ ]:
recommend_books_t_a_w

,title,author,image
0,Amazing Grace Lives of Children and the Conscience of a Nation The,Jonathan Kozol,</img
1,Assata An Autobiography Lawrence Hill amp Co,Assata Shakur,</img
2,Bayou Blood Brothers TylerNickJules Harlequin Intrigue 606,Rebecca York,</img
3,Brief Encounter Sealed With A Kiss Harlequin Romance No 3360,Catherine George,</img
4,Complete Chronicles of Narnia,C. S. Lewis,</img
5,Corinthian,Georgette Heyer,</img
6,Do You Take This Rebel The Calamity Janes Silhouette Special Edition No 1394,Janet Woods,</img
7,Dream Country Sandman Book 3,Neil Gaiman,</img
8,Expiration Date Men at Work,Aimee Thurlo,</img
9,Familiar Lullaby Fear Familiar Harlequin Intrigue No 614,Caroline Burnes,</img


In [ ]:
recommend('Fluir Flow Una Psicologia de La Felicidad')

[['A traves del tiempo',
  'Brian L. Weiss',
  'http://images.amazon.com/images/P/8466301240.01.MZZZZZZZ.jpg'],
 ['Acampar Manual Practico',
  'Antonio Ruiz Heredia',
  'http://images.amazon.com/images/P/8479550597.01.MZZZZZZZ.jpg'],
 ['Amar sin condiciones',
  'Louise L. Hay',
  'http://images.amazon.com/images/P/8479532076.01.MZZZZZZZ.jpg'],
 ['Amate a TI Mismo Cambiaras Tu Vida  Manual De Trabajo141',
  'Louise L. Hay',
  'http://images.amazon.com/images/P/8486344913.01.MZZZZZZZ.jpg'],
 ['Aprende FotografiaLearn Photography',
  'Antoine Desilets',
  'http://images.amazon.com/images/P/8423108562.01.MZZZZZZZ.jpg'],
 ['Aprendo a Decir No',
  'Ulrike Dahm',
  'http://images.amazon.com/images/P/8479272929.01.MZZZZZZZ.jpg'],
 ['AztecaAztec',
  'Gary Jennings',
  'http://images.amazon.com/images/P/8432037311.01.MZZZZZZZ.jpg'],
 ['Beltenebros Biblioteca breve',
  'Antonio Munoz Molina',
  'http://images.amazon.com/images/P/8432206008.01.MZZZZZZZ.jpg'],
 ['Bueno Para Comer',
  'Marvin Harris

In [ ]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ['The Fellowship of the Ring The Lord of the Rings Part 1',
  'J. R. R. Tolkien',
  'http://images.amazon.com/images/P/0618002227.01.MZZZZZZZ.jpg'],
 ['Stardust',
  'Neil Gaiman',
  'http://images.amazon.com/images/P/0380977281.01.MZZZZZZZ.jpg'],
 ['Martian Chronicles',
  'Ray Bradbury',
  'http://images.amazon.com/images/P/0380973839.01.MZZZZZZZ.jpg'],
 ['Neverwhere',
  'Neil Gaiman',
  'http://images.amazon.com/images/P/0380973634.01.MZZZZZZZ.jpg'],
 ['Stranger In the Mirror',
  'Sidney Sheldon',
  'http://images.amazon.com/images/P/0446341932.01.MZZZZZZZ.jpg'],
 ['Disclosure',
  'Michael Crichton',
  'http://images.amazon.com/images/P/0345391055.01.MZZZZZZZ.jpg'],
 ['Scandal in Fair Haven',
  'Carolyn G. Hart',
  'http://images.amazon.com/images/P/0553094653.01.MZZZZZZZ.jpg'],
 ['Silver Scream  A BedandBreakfast Mystery BedAndBreakfast Mysteries Paperback',
  'Mary Daheim',
  'htt

### Item-Based Collaborative Filterings

* **Item Based Collaborative Filtering** - Instead of measuring the similarity between users, the item-based CF recommends items based on their similarity with the items that the target user rated. Likewise, the similarity can be computed with Pearson Correlation or Cosine Similarity. The major difference is that, with item-based collaborative filtering, we fill in the blank vertically, as oppose to the horizontal manner that user-based CF does.

![](https://image.ibb.co/f6mDXU/conten.png)

### **Using Cosine Similarity**
![Screenshot%202023-05-14%20234705.png](attachment:Screenshot%202023-05-14%20234705.png)

![Screenshot%202023-05-14%20234840.png](attachment:Screenshot%202023-05-14%20234840.png)

In [ ]:
def item_based_recommender(title):
    
    book_title = str(title)
    if book_title in df['title'].values:
            
        user_book_df = df.pivot_table(index=['title'],
                                                    columns=['userid'],
                                                    values='rating').fillna(0)
        
        cosine_sim = cosine_similarity(user_book_df)
        index = np.where(user_book_df.index == book_title)[0][0]
        sim_books = list(enumerate(cosine_sim[index]))
        sorted_sim_books = sorted(sim_books,key=lambda x:x[1],
                                      reverse=True)[1:6]
            
        books = []
        for i in sorted_sim_books:
            books.append(user_book_df.index[i[0]])
        
        # Commenting out the image display code
        #fig = plt.figure(figsize=(18,5),dpi=100)
        #url = df.loc[df['title'] == book_title,'image_url_m'][:1].values[0]
        #im = Image.open(requests.get(url, stream=True).raw)
        #plt.imshow(im)
        #plt.axis('off')
        #plt.title('Rating: {}, \nAuthor: {}'\
        #               .format(round(df[df['title'] == book_title]['rating'].mean(),1),\
        #                       df['author'][df['title']== book_title].values[0]),\
        #                y=-0.20, color="black", fontsize=14, fontweight='bold', loc='left')
        #plt.show()
            
        print('Other books you may like:')
        for i in range(len(books)):
            print('{}, Author: {}'.format(books[i], df['author'][df['title']== books[i]].values[0]))
                
                     
    else:
        random = pd.Series(df['title'].unique()).sample(3).values
        print('Cant find book in dataset, please check spelling \n')
        print('Try: \n')
        print('{}'.format(random[0]),'\n')
        print('{}'.format(random[1]),'\n')
        print('{}'.format(random[2]),'\n')


In [ ]:
item_based_recommender('1984')

Other books you may like:
Animal Farm, Author: George Orwell
The Fellowship of the Ring The Lord of the Rings Part 1, Author: J. R. R. Tolkien
Stardust, Author: Neil Gaiman
Martian Chronicles, Author: Ray Bradbury
Neverwhere, Author: Neil Gaiman


In [ ]:
item_based_recommender('Final Fantasy Anthology Official Strategy Guide Brady Games')

Cant find book in dataset, please check spelling 

Try: 

Almost blue Stile libero 

Der Pferdeflsterer 

A Kiss Gone Bad 



### User Based Collaborative Filtering 

In [ ]:
def user_based_recommender(user_id):
    user_books = df[df['userid'] == user_id]
    user_books = user_books.drop_duplicates(subset=["title"], keep='first')
    sorted_books = user_books.sort_values(by='rating',ascending=False)
    top_books = sorted_books.head(10)
    return top_books

In [ ]:
user_id = int(input("Enter User id:"))
if user_id in df['userid'].values:
    try:
        user_recommend = pd.DataFrame(user_based_recommender(user_id),columns=['title', 'author','image_url_m'])
        user_recommend = user_recommend.style.format({'image_url_m':show_image})
    except IndexError:
        print("Enter the Title which is available")

Enter User id:254	


In [ ]:
user_recommend

,title,author,image_url_m
75,Neverwhere,Neil Gaiman,</img
52,American Gods,Neil Gaiman,</img
67,Corinthian,Georgette Heyer,</img
58,Yarrow,Charles de Lint,</img
95,Harlequin Valentine,Neil Gaiman,</img
93,Sandman The Dream Hunters,Neil Gaiman,</img
92,Dream Country Sandman Book 3,Neil Gaiman,</img
91,Assata An Autobiography Lawrence Hill amp Co,Assata Shakur,</img
87,Harry Potter and the Sorcerers Stone Book 1,J. K. Rowling,</img
81,Harry Potter and the Goblet of Fire Book 4,J. K. Rowling,</img


In [ ]:
user_based_recommender(12835)

,userid,rating,title,author,publisher,image_url_m,no_times_rated


In [ ]:
user_based_recommender(3757)

,userid,rating,title,author,publisher,image_url_m,no_times_rated
2434,3757,9,El Nombre De La RosaThe Name Of The Rose,Umberto Eco,Sites/Lumen Books,http://images.amazon.com/images/P/8426411487.0...,94
2406,3757,9,The Subtle Knife His Dark Materials Book 2,Philip Pullman,Listening Library,http://images.amazon.com/images/P/0807281859.0...,94
2412,3757,9,La Casa de los Espiritus,Isabel Allende,Continental Book Company,http://images.amazon.com/images/P/8401422248.0...,94
2411,3757,9,Cuentos de Eva Luna Plaza amp Jansliteraria,Isabel Allende,Plaza &amp; JanÃ©s,http://images.amazon.com/images/P/8401381630.0...,94
2410,3757,9,Eva Luna Plaza amp Jansliteraria,Isabel Allende,Ediciones del Norte,http://images.amazon.com/images/P/840138110X.0...,94
2408,3757,9,Mi Pais Inventado,Isabel Allende,Arete,http://images.amazon.com/images/P/8401341744.0...,94
2477,3757,9,Cuentos de hadas retorcidos,A.J. Jacobs,Sirio,http://images.amazon.com/images/P/8478082875.0...,94
2492,3757,9,Platero y Yo,Juan Ramon Jimenez,"Editores Mexicanos Unidos, S.A.",http://images.amazon.com/images/P/9681500601.0...,94
2428,3757,8,Gua prctica del peregrino El Camino de Santiago,MillÃ¡n Bravo Lozano,Editorial Everest,http://images.amazon.com/images/P/8424139070.0...,94
2429,3757,8,Tus Zonas ErroneasYour Erroneous Zones,Wayne W. Dyer,Lectorum Pubns,http://images.amazon.com/images/P/8425309506.0...,94


In [ ]:
def user_based_recommender(user_id):
    
    if user_id in df['userid'].values:
        user_df = df.pivot_table(index=['userid'], columns=['title'], values='rating').fillna(0)
        cosine_sim = cosine_similarity(user_df)
        user_sim_df = pd.DataFrame(cosine_sim)
        # Set the index and column name to user Ids
        user_sim_df.index = list(user_df.index)
        user_sim_df.columns = list(user_df.index)
        index = np.where(user_sim_df.index == user_id)[0][0]
        sim_books = list(enumerate(user_sim_df[user_sim_df.index[index]]))
        sorted_sim_books = sorted(sim_books,key=lambda x:x[1], reverse=True)[1:50]
        users = []
        for i in sorted_sim_books:
            users.append(user_sim_df.index[i[0]])
        book_list = []
        for i in users:
            book_list = book_list+list(df[df['userid']==i]['title'])
        final_books = sorted(set(book_list)-set(df[df['userid']==user_id]['title']))
        print("Top 5 Recommended books for User ID", user_id)
        print("---------------------------")
        for i, book in enumerate(final_books[:5]):
            print(i+1, ".", book)


In [ ]:
user_based_recommender(3757)

Top 5 Recommended books for User ID 3757
---------------------------
1 .  Sie belieben wohl zu scherzen Mr Feynman Abenteuer eines neugierigen Physikers
2 . 010100 A Novel of the Millennium
3 . 010100 The Novel of the Millennium
4 . 1 2 3 Music
5 . 10 Minute Guide to HTML 32


In [ ]:
user_based_recommender(12835)

### Content-Based Collaborative Filterings
In this recommender system the content of the book (Author) is used to find its similarity with other books. Then the books that are most likely to be similar are recommended. 

### Author Based

In [ ]:
def author_based_recommender(author_name):
    author_books = df[df['author'] == author_name]
    author_books = author_books.drop_duplicates(subset=["title", "publisher"], keep='first')
    sorted_books = author_books.sort_values(by='rating',ascending=False)
    top_books = sorted_books.head(10)
    return top_books

In [ ]:
author_name = input("Enter author_name:")
if df['author'].str.contains(author_name).any():
    try:
        recommend_books_A = pd.DataFrame(author_based_recommender(author_name),columns=['title', 'author','image_url_m'])
        recommend_books_A = recommend_books_A.style.format({'image_url_m':show_image})
    except IndexError:
        print("Enter the Title which is available")

Enter author_name:Wally Lamb	


In [ ]:
def author_based_recommender(author_name):
    author_books = df[df['author'] == author_name]['title'].unique()
    author_ratings = df[df['title'].isin(author_books)].groupby('title')['rating'].mean().sort_values(ascending=False)
    top_books = author_ratings.index[:5]
    print("Top 5 Recommended books by", author_name)
    print("---------------------------")
    for i, book in enumerate(top_books):
        print(i+1, ".", book)


In [ ]:
author_counts = df['author'].value_counts()
print(author_counts)

Stephen King       61
James Patterson    52
Janet Evanovich    51
Nora Roberts       50
Agatha Christie    48
                   ..
Larissa Lai         1
Charles De Lint     1
JUNG CHANG          1
Norah Lofts         1
Maura Spiegel       1
Name: author, Length: 5065, dtype: int64


In [ ]:
author_based_recommender('Mihaly Csikszentmihalyi')

Top 5 Recommended books by Mihaly Csikszentmihalyi
---------------------------
1 . Fluir Flow Una Psicologia de La Felicidad


In [ ]:
author_based_recommender('Stephen King')

Top 5 Recommended books by Stephen King
---------------------------
1 . Wizard and Glass The Dark Tower Book 4
2 . The Stand The Complete amp Uncut Edition
3 . From a Buick 8
4 . The Girl Who Loved Tom Gordon
5 . It R


### Comparison of All Recommenders

In [ ]:
item_based_recommender('1984')

Other books you may like:
Animal Farm, Author: George Orwell
The Fellowship of the Ring The Lord of the Rings Part 1, Author: J. R. R. Tolkien
Stardust, Author: Neil Gaiman
Martian Chronicles, Author: Ray Bradbury
Neverwhere, Author: Neil Gaiman


In [ ]:
book_title = '1984'
book_data = df[df['title'] == book_title]
author_name = book_data['author'].values[0]
user_ids = book_data['userid'].unique()

print("Author of '{}' book: {}".format(book_title, author_name))
print("User IDs that recommend '{}' book:".format(book_title))
print(user_ids)

Author of '1984' book: George Orwell
User IDs that recommend '1984' book:
[  254  7346 11676]


In [ ]:
author_based_recommender('George Orwell')

Top 5 Recommended books by George Orwell
---------------------------
1 . 1984
2 . Animal Farm
3 . La Ferme Des Animaux


In [ ]:
user_based_recommender(90232)

### Conclusion 
We create recommenders using demographic , content- based and collaborative filtering. While demographic filtering is very elemantary and cannot be used practically, Hybrid Systems can take advantage of content-based and collaborative filtering as the two approaches are proved to be almost complimentary. This model was very baseline and only provides a fundamental framework to start with.

I would like to mention some excellent refereces that I learned from